In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [ ]:
train_data = pd.read_csv('train.csv')
labels = []
pixels =[]
for index, row in train_data.iterrows():
    label=np.zeros(10)
    label[row["label"]]=1
    labels.append(label)
    pixels.append(row[1:])
labels = np.array(labels)
pixels = np.array(pixels)

In [ ]:
test_data = pd.read_csv('test.csv')
test_pixels = np.array(test_data)
print(test_pixels.shape)
test_pixels[0].shape

In [ ]:
# Initializing learning parameters
lr = 0.0003
steps = 420000
batch_size = 50
sess = tf.InteractiveSession()

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

x = tf.placeholder('float',[None,784],name="x")
y_ = tf.placeholder('float',[None,10],name="y_")

In [ ]:
W_conv1 = weight_variable([2, 2, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [ ]:
end = 0
def next_training_batch(batch_size):
    global end
    start = end
    end += batch_size
    if end > len(pixels):
        difference = batch_size - (end - len(pixels))
        return_img = np.concatenate((pixels[end-batch_size:],pixels[:difference]))
        return_labels=np.concatenate((labels[end-batch_size:],labels[:difference]))
        end = difference
    else:
        return_img=pixels[start:end]
        return_labels = labels[start:end]
    return (return_img,return_labels)

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(steps):
    batch = next_training_batch(batch_size)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

In [ ]:
saver = tf.train.Saver()
save_path = saver.save(sess, "tmp/model.ckpt")
print("Model saved in path: %s" % save_path)

In [ ]:
#new_saver = tf.train.import_meta_graph('tmp/new1.ckpt.meta')
#new_saver.restore(sess,"tmp/new1.ckpt")
test_results= []
test_results= np.array(test_results)
feed_dict = {x:test_pixels,keep_prob : 1.0}
test_results = sess.run(tf.argmax(y_conv,1),feed_dict)
print(test_results)

In [ ]:
print(len(test_results))

In [ ]:
result_values = pd.Series(test_results)
index_values = pd.Series(range(1,28001))
result_df=pd.DataFrame(dict(ImageId = index_values, Label = result_values))

In [ ]:
result_df.to_csv('submission.csv',index=False)

In [ ]:
files.download('submission.csv')